centroid connector builder
description: generate centroid connectors for OSM drive network
Sijia.Wang@wsp.com
Feb 7, 2019

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, shape, LineString
from scipy.spatial import cKDTree

In [2]:
#run option
run_drive = True
run_walk = True
#define input files
#osm drive network (edge/node), centroid connector file, node coordinates
cd = "C:/Users/wangs1/Documents/Met_Council/"
osm_drive_edge_file = cd + "data/network-shape_MetCouncil_full/osm_networktype_drive/consolidate_lane/osm_complete_lanes.shp"
osm_drive_node_file = cd + "data/network-shape_MetCouncil_full/osm_networktype_drive/drive_node.shp"
osm_walk_edge_file = cd + "data/network-shape_MetCouncil_full/osm_networktype_walk/walk_edge.shp"
osm_walk_node_file = cd + "data/network-shape_MetCouncil_full/osm_networktype_walk/walk_node.shp"
centroid_connector_file = cd + "centroid_connector_builder/centroid_links.csv"
node_coordinates_file = cd + "centroid_connector_builder/node_coordinates.csv"

In [3]:
#read input files
osm_drive_edge_gdf = gpd.read_file(osm_drive_edge_file)
osm_drive_node_gdf = gpd.read_file(osm_drive_node_file)
osm_walk_edge_gdf = gpd.read_file(osm_walk_edge_file)
osm_walk_node_gdf = gpd.read_file(osm_walk_node_file)
centroid_connector_df = pd.read_csv(centroid_connector_file)
node_coordinates_df = pd.read_csv(node_coordinates_file)

In [17]:
osm_drive_node_gdf.shape

(167538, 8)

In [13]:
#define key param
#zone_hi = 3030
external_hi = 3061

In [4]:
osm_drive_edge_gdf = osm_drive_edge_gdf.to_crs(epsg = 26915)
osm_drive_node_gdf = osm_drive_node_gdf.to_crs(epsg = 26915)
osm_walk_edge_gdf = osm_walk_edge_gdf.to_crs(epsg = 26915)
osm_walk_node_gdf = osm_walk_node_gdf.to_crs(epsg = 26915)

In [5]:
osm_walk_edge_gdf.columns = [x.upper() if x is not 'geometry' else x for x in osm_walk_edge_gdf.columns]

In [ ]:
#osm_node_gdf = osm_node_gdf.to_crs(epsg = 26915)
#osm_node_gdf['x_reproj'] = osm_node_gdf['geometry'].apply(lambda p: p.x)
#osm_node_gdf['y_reproj'] = osm_node_gdf['geometry'].apply(lambda p: p.y)

In [6]:
centroid_connector_df['c'] = centroid_connector_df[['A', 'B']].min(axis = 1)
centroid_connector_df['non_c'] = centroid_connector_df[['A', 'B']].max(axis = 1)

In [7]:
def num_of_drive_loadpoint_per_centroid(existing_drive_cc_df, node_coordinates_df):
    existing_pairs_of_centroid_loadpoint_df = existing_drive_cc_df.groupby(['c', 'non_c']).count().reset_index().drop(['A','B'], axis = 1)
    existing_num_of_loadpoint_per_c_df = existing_drive_cc_df.groupby(['c', 'non_c']).count().groupby('c').count()[['A']].rename(columns = {'A':'abm_num_load'}).reset_index()
    num_drive_loadpoint_new_near_old = pd.merge(existing_pairs_of_centroid_loadpoint_df,
                                                        existing_num_of_loadpoint_per_c_df,
                                                        how = 'left',
                                                        on = 'c')
    num_drive_loadpoint_new_near_old['osm_num_load'] = (4 / num_drive_loadpoint_new_near_old['abm_num_load']).astype(int)
    num_drive_loadpoint_new_near_old = pd.merge(num_drive_loadpoint_new_near_old,
                                                        node_coordinates_df[['N', 'X', 'Y']],
                                                        how = 'left',
                                                        left_on = 'non_c',
                                                        right_on = 'N')
    return num_drive_loadpoint_new_near_old

def num_of_walk_loadpoint_per_centroid(node_coordinates_df):
    num_walk_loadpoint = node_coordinates_df[['N', 'X', 'Y']].copy()
    num_walk_loadpoint = num_walk_loadpoint[num_walk_loadpoint['N'] <= external_hi]
    num_walk_loadpoint['osm_num_load'] = np.int(5)
    num_walk_loadpoint.rename(columns = {'N':'c'}, inplace = True)
    
    return num_walk_loadpoint
#develop_new_cc_df = pd.merge(existing_drive_cc_df.groupby(['c', 'non_c']).count().reset_index().drop(['A','B'], axis = 1),
#         existing_drive_cc_df.groupby(['c', 'non_c']).count().groupby('c').count()[['A']].rename(columns = {'A':'abm_num_load'}).reset_index(),
#         how = 'left', 
#         on = 'c')
#develop_new_cc_df['osm_num_load'] = (4 / develop_new_cc_df['abm_num_load']).astype(int)
#develop_new_cc_df = pd.merge(develop_new_cc_df, 
#                            node_coordinates_df[['N', 'X', 'Y']],
#                            how = 'left',
#                            left_on = 'non_c',
#                            right_on = 'N')

In [8]:
def find_new_load_point(abm_load_ref_df, osm_node_gdf):
    osm_node_gdf['x_reproj'] = osm_node_gdf['geometry'].apply(lambda p: p.x)
    osm_node_gdf['y_reproj'] = osm_node_gdf['geometry'].apply(lambda p: p.y)
    inventory_node_ref = osm_node_gdf[['x_reproj', 'y_reproj']].values
    tree = cKDTree(inventory_node_ref)
    new_load_point_gdf = gpd.GeoDataFrame()
    for i in range(len(abm_load_ref_df)):
        point = abm_load_ref_df.iloc[i][['X', 'Y']].values
        n_neigh = abm_load_ref_df.iloc[i]['osm_num_load']
        dd, ii = tree.query(point, k = n_neigh)
        if n_neigh == 1:
            add_gdf = gpd.GeoDataFrame(osm_node_gdf[['osmid', 'geometry']].iloc[ii]).transpose().reset_index(drop = True)
        else:
            add_gdf = gpd.GeoDataFrame(osm_node_gdf[['osmid', 'geometry']].iloc[ii]).reset_index(drop = True)
        add_gdf['c'] = int(abm_load_ref_df.iloc[i]['c'])
        if i == 0:
            new_load_point_gdf = add_gdf.copy()
        else:
            new_load_point_gdf = new_load_point_gdf.append(add_gdf, ignore_index=True, sort=False)
        #print(i)
        #print(add_gdf)
    return new_load_point_gdf.rename(columns = {'geometry' : 'geometry_ld'})

In [9]:
def generate_centroid_connectors(run_type, existing_drive_cc_df, osm_node_gdf, abm_node_df):
    if run_type == 'drive':
        abm_load_ref_df = num_of_drive_loadpoint_per_centroid(existing_drive_cc_df, abm_node_df)
    if run_type == 'walk':
        abm_load_ref_df = num_of_walk_loadpoint_per_centroid(abm_node_df)

    new_load_point_gdf = find_new_load_point(abm_load_ref_df, osm_node_gdf)
    new_load_point_gdf = pd.merge(new_load_point_gdf,
                                 abm_node_df[['N', 'X', 'Y']],
                                 how = 'left', 
                                 left_on = 'c',
                                 right_on = 'N')
    new_load_point_gdf['geometry_c'] = [Point(xy) for xy in zip(new_load_point_gdf['X'], new_load_point_gdf['Y'])]
    new_load_point_gdf.drop(['N', 'X', 'Y'], axis = 1, inplace = True)
    
    #centroid coordinates
    new_centroid_gdf = new_load_point_gdf.copy()[['c', 'geometry_c']]
    new_centroid_gdf.rename(columns = {'c' : 'osmid', 'geometry_c' : 'geometry'}, inplace = True)
    new_centroid_gdf.drop_duplicates(['osmid'], inplace = True)
    
    #inbound cc
    new_cc_in_gdf = new_load_point_gdf.copy()
    new_cc_in_gdf['geometry'] = [LineString(xy) for xy in zip(new_cc_in_gdf['geometry_ld'], new_cc_in_gdf['geometry_c'])]
    new_cc_in_gdf = new_cc_in_gdf.rename(columns = {'osmid' : 'osmid_from', 'c' : 'osmid_to'})
    
    #outbound cc
    new_cc_out_gdf = new_load_point_gdf.copy()
    new_cc_out_gdf['geometry'] = [LineString(xy) for xy in zip(new_cc_out_gdf['geometry_c'], new_cc_out_gdf['geometry_ld'])]
    new_cc_out_gdf = new_cc_out_gdf.rename(columns = {'osmid' : 'osmid_to', 'c' : 'osmid_from'})
    
    #bind
    new_cc_gdf = new_cc_in_gdf.append(new_cc_out_gdf, ignore_index = True, sort = False)
    new_cc_gdf.drop(['geometry_ld', 'geometry_c'], axis = 1, inplace = True)
    
    #remove duplicates
    new_cc_gdf.drop_duplicates(['osmid_from', 'osmid_to'], inplace = True)
    
    new_cc_gdf.crs = {'init' : 'epsg:26915'}
    new_centroid_gdf.crs = {'init' : 'epsg:26915'}
    
    return new_cc_gdf, new_centroid_gdf

In [10]:
def format_and_export(run_type, new_cc_gdf, osm_edge_gdf, new_centroid_gdf, osm_node_gdf):
    #new_cc_gdf.crs = {'init' : 'epsg:26915'}
    #osm_edge_gdf = osm_edge_gdf.to_crs(epsg = 26915)
    new_cc_gdf['LANES_CONS'] = 1
    if ('LANES_CONS' in osm_edge_gdf) == False:
        osm_edge_gdf['LANES_CONS'] = 1
    new_cc_gdf['CENTROID'] = int(1)
    osm_edge_gdf['CENTROID'] = int(0)
    osm_edge_gdf.rename(columns = {'U' : 'osmid_from', 'V' : 'osmid_to'}, inplace = True)
    osm_edge_with_cc_gdf = osm_edge_gdf.append(new_cc_gdf, ignore_index = True, sort = False)
    osm_edge_with_cc_gdf['LINK_ID'] = range(1, 1+len(osm_edge_with_cc_gdf))
    osm_edge_with_cc_gdf.to_file(cd + "/centroid_connector_builder/osm_" + run_type + "_edges_with_cc_new.shp")
    
    osm_node_with_centroid_gdf = osm_node_gdf.append(new_centroid_gdf, ignore_index = True, sort = False)
    osm_node_with_centroid_gdf['osmid'] = pd.to_numeric(osm_node_with_centroid_gdf['osmid'], downcast = 'integer')
    #osm_node_with_centroid_gdf.crs = {'init' : 'epsg:26915'}
    
    #create consecutive node id
    osm_node_reindex_gdf = osm_node_with_centroid_gdf[['osmid', 'geometry']].copy()
    osm_node_reindex_gdf.sort_values(by = ['osmid'], inplace = True)
    osm_node_reindex_gdf['N'] = range(1, 1+len(osm_node_reindex_gdf))
    
    #create X, Y
    osm_node_reindex_gdf['X'] = osm_node_reindex_gdf['geometry'].apply(lambda p: p.x)
    osm_node_reindex_gdf['Y'] = osm_node_reindex_gdf['geometry'].apply(lambda p: p.y)
    osm_node_reindex_gdf.rename(columns = {'osmid' : 'OSMID'}, inplace = True)
    
    #save output
    pd.DataFrame(osm_node_reindex_gdf[['N', 'X', 'Y', 'OSMID']]).to_csv(cd + "/centroid_connector_builder/" + run_type + "_node.csv",
                                                                      index = False)
    osm_node_reindex_gdf[['N', 'X', 'Y', 'OSMID', 'geometry']].to_file(cd + "/centroid_connector_builder/" + run_type + "_node.shp")
    
    return osm_edge_with_cc_gdf, osm_node_reindex_gdf

In [11]:
if run_drive == True:
    new_drive_cc_gdf, new_drive_centroid_gdf = generate_centroid_connectors('drive',
                                                                        centroid_connector_df,
                                                                        osm_drive_node_gdf, 
                                                                        node_coordinates_df)
    osm_drive_edge_with_cc_gdf, osm_drive_node_with_c_gdf = format_and_export('drive', 
                                                                              new_drive_cc_gdf, 
                                                                              osm_drive_edge_gdf,
                                                                              new_drive_centroid_gdf,
                                                                              osm_drive_node_gdf)

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [14]:
if run_walk == True:
    new_walk_cc_gdf, new_walk_centroid_gdf = generate_centroid_connectors('walk',
                                                                        centroid_connector_df,
                                                                        osm_walk_node_gdf, 
                                                                        node_coordinates_df)
    osm_walk_edge_with_cc_gdf, osm_walk_node_with_c_gdf = format_and_export('walk', 
                                                                            new_walk_cc_gdf,
                                                                            osm_walk_edge_gdf, 
                                                                            new_walk_centroid_gdf,
                                                                            osm_walk_node_gdf)

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [15]:
osm_drive_node_with_c_gdf[['N', 'X', 'Y', 'OSMID', 'geometry']].to_file(cd + "/centroid_connector_builder/" + 'drive' + "_node.shp")
osm_drive_node_with_c_gdf[['N', 'X', 'Y', 'OSMID', 'geometry']].to_file(cd + "/centroid_connector_builder/" + 'bike' + "_node.shp")
osm_walk_node_with_c_gdf[['N', 'X', 'Y', 'OSMID', 'geometry']].to_file(cd + "/centroid_connector_builder/" + 'walk' + "_node.shp")

osm_drive_node_with_c_gdf['isDriveNode'] = np.int(1)
osm_bike_node_with_c_gdf = osm_drive_node_with_c_gdf.copy()
osm_bike_node_with_c_gdf['isBikeNode'] = np.int(1)
osm_walk_node_with_c_gdf['isWalkNode'] = np.int(1)

node_all_gdf = pd.concat([osm_drive_node_with_c_gdf, osm_bike_node_with_c_gdf, osm_walk_node_with_c_gdf],
                         ignore_index=True, sort = False)
node_all_gdf.to_csv(cd + "/centroid_connector_builder/all_nodes_with_centroid.csv",
                                                                      index = False)